In [324]:
from nltk.corpus import wordnet as wn
from lxml import etree
import numpy as np
import pandas as pd
from os import path
from collections import Counter
from copy import deepcopy

In [325]:
taxonomy = pd.read_csv('taxonomy.csv', header=0)
taxonomy.columns

Index(['alco-soft', 'alco-soft-bw', 'alco', 'noalco-fruits-juices-j_sweet',
       'noalco-fruits-juices-j_citric', 'noalco-fruits-solid-s_sweet',
       'noalco-fruits-solid-s_citric', 'noalco-veggies',
       'noalco-fruits-syrups-j_sweet', 'te-sweet', 'te', 'others',
       'Unnamed: 12'],
      dtype='object')

In [355]:
global taxonomy_scheme
global all_ingredients
global df_pmi
global df_preferences
global taxonomy_general_cat
taxonomy_scheme = {
    ('alco', 'soft', 'bw'): [],
    ('alco', 'soft'): [],
    ('alco', ): [],
    ('noalco', 'fruits', 'juices', 'j_sweet'): [],
    ('noalco', 'fruits', 'juices', 'j_citric'): [],
    ('noalco', 'fruits', 'solid', 's_sweet'): [],
    ('noalco', 'fruits', 'solid', 's_citric'): [],
    ('noalco', 'veggies'): [],
    ('noalco', 'fruits', 'syrups', 'j_sweet'): [],
    ('te', 'sweet'): [],
    ('te',): [],
    ('others',): []
}

taxonomy_general_cat = [
 'alco', 'noalco', 'te', 'others'
]

In [327]:
light_martini = {'Light cocktail martini': {
   ('alco', 'soft', 'bw'): [],
   ('alco', 'soft'): [],
   ('alco', ): ['red martini'],
   ('noalco', 'fruits', 'juices', 'j_sweet'): [],
   ('noalco', 'fruits', 'juices', 'j_citric'): [],
   ('noalco', 'fruits', 'solid', 's_sweet'): [],
   ('noalco', 'fruits', 'solid', 's_citric'): ['lime'],
   ('noalco', 'veggies'): [],
   ('noalco', 'fruits', 'syrups', 'j_sweet'): [],
   ('te', 'sweet'): ['sugar'],
   ('te',): [],
   ('others',): ['sparkling mineral water']}
}

In [328]:
from copy import deepcopy
def read_data():
    # read
    taxonomy = pd.read_csv('taxonomy.csv', header=0)
    tree = etree.parse('ccc_cocktails.xml')

    recipes = tree.findall('recipe')
    titles = []
    all_ingredients = []
    cocktails = {}
    for recipe in recipes:
        title = recipe[0].text
        titles.append(title)
        ingredients = recipe[1]
        ingredients_list = deepcopy(taxonomy_scheme)
        for ingredient in ingredients:
            ingredient_val = ingredient.values()[-1].lower()
            all_ingredients.append(ingredient_val)
            for col in taxonomy.columns:
                col_tup = tuple(col.split('-'))

                if taxonomy[col].isin([ingredient_val]).any():
                    ingredients_list[col_tup].append(ingredient_val)

        cocktails[title] = ingredients_list
    return all_ingredients, cocktails

In [329]:
all_ingredients, cocktails = read_data()

## Similarities preferences

In [330]:
cols = [
        
    ('alco', 'soft', 'bw'),
    ('alco', 'soft'),
    ('alco', ),
    ('noalco', 'fruits', 'juices', 'j_sweet'),
    ('noalco', 'fruits', 'juices', 'j_citric'),
    ('noalco', 'fruits', 'solid', 's_sweet'),
    ('noalco', 'fruits', 'solid', 's_citric'),
    ('noalco', 'veggies'),
    ('noalco', 'fruits', 'syrups', 'j_sweet'),
    ('te', 'sweet'),
    ('te',), 
    ('others',)
]
idx = ['alco', 'sweet', 'sour', 'tutti-frutti', 'default']

alco_vals =  [0.85, 0.9, 1, 0.7, 0.7, 0.7, 0.7, 0.75, 0.75, 0.7, 0.8, 0.85]

sweet_vals = [0.9, 0.95, 0.8, 0.85, 0.5, 0.9, 0.65, 0.5, 1, 1, 0.8, 0.8]

sour_vals = [0.8, 0.6, 0.8, 0.7, 1, 0.75, 1, 0.8, 0.6, 0.6, 0.85, 0.8]

frutti_vals = [0.85, 0.9, 0.7, 1, 1, 1, 1, 0.7, 0.9, 0.65, 0.7, 0.85]

default = [1,1,1,1,1,1,1,1,1,1,1,1]
df_preferences = pd.DataFrame(np.array([alco_vals, sweet_vals, sour_vals, frutti_vals, default]), columns=cols, index=idx)
df_preferences

,"(alco, soft, bw)","(alco, soft)","(alco,)","(noalco, fruits, juices, j_sweet)","(noalco, fruits, juices, j_citric)","(noalco, fruits, solid, s_sweet)","(noalco, fruits, solid, s_citric)","(noalco, veggies)","(noalco, fruits, syrups, j_sweet)","(te, sweet)","(te,)","(others,)"
alco,0.85,0.90,1.0,0.70,0.7,0.70,0.70,0.75,0.75,0.70,0.80,0.85
sweet,0.90,0.95,0.8,0.85,0.5,0.90,0.65,0.50,1.00,1.00,0.80,0.80
sour,0.80,0.60,0.8,0.70,1.0,0.75,1.00,0.80,0.60,0.60,0.85,0.80
tutti-frutti,0.85,0.90,0.7,1.00,1.0,1.00,1.00,0.70,0.90,0.65,0.70,0.85
default,1.00,1.00,1.0,1.00,1.0,1.00,1.00,1.00,1.00,1.00,1.00,1.00


In [331]:
df_preferences[('noalco', 'fruits', 'solid', 's_citric')]['alco']

0.7

## PMI table

In [332]:
def constract_pmi_table():
    unique_ingredients = set(all_ingredients)
    df_pmi = pd.DataFrame(0, columns=unique_ingredients, index=unique_ingredients)
    for recipe in recipes:
        title = recipe[0].text
        titles.append(title)
        ingredients = recipe[1]
        for ingredient_row in ingredients:
            ingredient_val_row = ingredient_row.values()[-1].lower()
            for ingredient_col in ingredients:
                ingredient_val_col = ingredient_col.values()[-1].lower()
                if ingredient_val_row != ingredient_val_col:
                    df_pmi.loc[ingredient_val_row, ingredient_val_col] += 1
    return df_pmi

In [333]:
df_pmi = constract_pmi_table()

In [334]:
from math import log
def pmi_score(ingredient1, ingredient2, df_pmi):
    fij = df_pmi.loc[ingredient1, ingredient2]
    fi = df_pmi.loc[ingredient1, :].sum()
    fj = df_pmi.loc[ingredient2, :].sum()
    sum_all = df_pmi.values.sum()
    print('fij', fij)
    print('fifj', fi*fj)
    p = fij/(fi*fj/sum_all)
    '''
    pmi = log(p, 2)
    if pmi < 0:
        pmi = 0
    return pmi, p
    '''
    return p
print('PMI table', df_pmi.loc['tequila', 'lemon'])
pmi_score('tequila', 'lime', df_pmi)

PMI table 0
fij 1
fifj 1121


1.6931311329170384

In [335]:
def get_cocktail_ingredient_list(cocktail):
    cocktail = cocktails[title]
    ingredient_list = []
    categories_list = []
    for cat in taxonomy_scheme.keys():
        for ingredient in cocktail[cat]:
            categories_list.append(cat)
            ingredient_list.append(ingredient)
    return categories_list, ingredient_list

In [336]:
from itertools import combinations
def get_cocktail_pmi(cocktail, preference=''):
    # cocktail pmi
    catetgory_list, ingridient_list = get_cocktail_ingredient_list(cocktail)
    pmi_score = 0
    c_combos = 0
    
    for ingredient1_tup, ingredient2_tup in combinations(zip(catetgory_list, ingridient_list), 2):
        ingredient1 = ingredient1_tup[1]
        ingredient2 = ingredient2_tup[1]
        if preference:
            cat1 = ingredient1_tup[0]
            cat2 = ingredient2_tup[0]
            w = (df_preferences[cat1][preference] + df_preferences[cat2][preference])/2
        else:
            w = 1
        pmi_score += df_pmi.loc[ingredient1, ingredient2] * w
        c_combos += 1
    return pmi_score/c_combos
print(get_cocktail_pmi(light_martini, preference='tutti-frutti'))
print(get_cocktail_pmi(light_martini, preference=''))

1.2675
1.6


In [152]:
light_martini

{'Light cocktail martini': {('liquid', 'alcohol', 'soft', 'bw'): [],
  ('liquid', 'alcohol', 'soft'): [],
  ('liquid', 'alcohol'): ['red martini'],
  ('liquid', 'juices'): [],
  ('liquid', 'juices', 'citric'): [],
  ('solid', 'fruits'): [],
  ('solid', 'fruits', 'citric'): ['lime'],
  ('solid', 'veggies'): ['mint'],
  ('te', 'syrups'): [],
  ('te', 'sweet'): ['sugar'],
  ('te',): [],
  ('others',): ['sparkling mineral water']}}

## Similarity in taxonomy

In [337]:
# ingredient_tup => (cat1, cat2, cat3, ingredient)
def taxonomy_score(ingredient_tup1, ingredient_tup2):
    common_ancestors = set(ingredient_tup1).intersection(set(ingredient_tup2))
    all_ancestors = set(ingredient_tup1).union(set(ingredient_tup2))
    return len(common_ancestors)/len(all_ancestors)

In [338]:
print(taxonomy_score(('alcohol', 'vodka'), ('alcohol', 'soft', 'malibu')))
print(taxonomy_score(('noalco', 'fruits', 'juices', 'j_citric', 'lemon juice'), ('noalco', 'veggie', 'mint')))

0.25
0.14285714285714285


## Match cocktails (retrieval)

In [349]:
q= {
    ('alco', 'soft', 'bw'): [],
    ('alco', 'soft'): [],
    ('alco', ): ['red martini'],
    ('noalco', 'fruits', 'juices', 'j_sweet'): [],
    ('noalco', 'fruits', 'juices', 'j_citric'): ['lemon juice'],
    ('noalco', 'fruits', 'solid', 's_sweet'): [],
    ('noalco', 'fruits', 'solid', 's_citric'): ['lime', 'orange'],
    ('noalco', 'veggies'): [],
    ('noalco', 'fruits', 'syrups', 'j_sweet'): [],
    ('te', 'sweet'): ['sugar'],
    ('te',): [],
    ('others',): []
}

In [368]:
def transform(cocktail):
    categories = list(taxonomy_scheme.keys())
    new_form = {}
    for cat in categories:
        for ingredient in cocktail[cat]:
            ingredient_cat = tuple(list(cat) + [ingredient])
            new_form.setdefault(cat[0], []).append(ingredient_cat)
        if not cocktail[cat]:
            new_form.setdefault(cat[0], [])
    return new_form
transform(light_martini['Light cocktail martini'])

{'alco': [('alco', 'red martini')],
 'noalco': [('noalco', 'fruits', 'solid', 's_citric', 'lime')],
 'te': [('te', 'sweet', 'sugar')],
 'others': [('others', 'sparkling mineral water')]}

In [369]:
transform(q)

{'alco': [('alco', 'red martini')],
 'noalco': [('noalco', 'fruits', 'juices', 'j_citric', 'lemon juice'),
  ('noalco', 'fruits', 'solid', 's_citric', 'lime'),
  ('noalco', 'fruits', 'solid', 's_citric', 'orange')],
 'te': [('te', 'sweet', 'sugar')],
 'others': []}

In [342]:
def exactMatchCocktails(q_ingredients_list, cocktails, preference='alco'):
    res = {}
    categories = list(taxonomy_scheme.keys())
    for title in cocktails.keys():
        cocktail = cocktails[title]
        
        matches = deepcopy(taxonomy_scheme)
        c_matches = 0
        sim_matches = 0
        for cat in categories:
            for q_ingredient in q_ingredients_list[cat]:
                if q_ingredient in cocktail[cat]:
                    matches[cat] = q_ingredient
                    sim_matches += df_preferences[cat][preference]
            
        res[title] = (sim_matches, matches)
    return res

In [343]:
def matchCocktails(q_ingredients_list, cocktails, preference='alco'):
    res = {}
    categories = list(taxonomy_scheme.keys())
    for title in cocktails.keys():
        cocktail = cocktails[title]
        
        matches = deepcopy(taxonomy_scheme)
        sim_matches = 0
        for cat in categories:
            #print('cat', cat)
            w = df_preferences[cat][preference]
            for q_ingredient in q_ingredients_list[cat]:
                q_ingredient_cat = tuple(list(cat) + [q_ingredient])
                c_matches = 0
                #print('q', q_ingredient)
                for d_ingredient in cocktail[cat]:
                    #print('d', d_ingredient)
                    d_ingredient_cat = tuple(list(cat) + [d_ingredient])
                    sim_matches += (taxonomy_score(q_ingredient_cat, d_ingredient_cat) * w)
                    c_matches += 1
                #print('sim', taxonomy_score(q_ingredient_cat, d_ingredient_cat))
                #print('c', c_matches)
                if c_matches:
                    sim_matches /= c_matches
            #print()
        res[title] = (sim_matches, cocktail)

    return res

In [390]:
def matchCocktails2(q_ingredients_list, cocktails, preference='alco'):
    res = {}
    categories = list(taxonomy_scheme.keys())
    for title in cocktails.keys():
        cocktail = cocktails[title]
        cocktail_form = transform(cocktail)
        query_form = transform(q_ingredients_list)
        sim = 0
        for gen_cat in taxonomy_general_cat:
            for q_ingredient_cat in query_form[gen_cat]:
                sim_gen_cat = 0
                c_gen_cat = 0
                # if the same ingredient exists in the cocktail
                # assign a similarity of one and don't investigate further
                if q_ingredient_cat in cocktail_form[gen_cat]:
                    sim_gen_cat = 1
                    c_gen_cat = 1
                else:
                    for d_ingredient_cat in cocktail_form[gen_cat]:
                        q_cat = q_ingredient_cat[:-1]
                        d_cat = d_ingredient_cat[:-1]
                        w = (df_preferences[q_cat][preference] + df_preferences[d_cat][preference])/2
                        sim_gen_cat += (taxonomy_score(q_ingredient_cat, d_ingredient_cat) * w)
                        c_gen_cat += 1
                if c_gen_cat:
                    sim += (sim_gen_cat/c_gen_cat)
        res[title] = (sim, cocktail) 
    return res

In [392]:
def get_matches(q, cocktails, top_n=2):
    #matches = exactMatchCocktails(q, cocktails)
    matches = matchCocktails2(q, cocktails)
    titles_sort = sorted(matches, key=lambda x: matches[x][0], reverse=True )
    max_val = matches[titles_sort[0]][0]
    final_match = {}
    sim_match = {}
    #top_n = 4
    c_top = 0
    for title in titles_sort:
        if c_top < top_n: #matches[title][0] == max_val or
            final_match[title] = matches[title][1]
            sim_match[title] = matches[title][0]
            c_top += 1
    return sim_match, final_match

In [393]:
sim_match, cocktail_matches = get_matches(q, cocktails, top_n=3)
sim_match, cocktail_matches

({'Light cocktail martini': 3.424404761904762,
  'Cocktail appetizer': 2.9008333333333334,
  'Strawberry Margarita': 2.8916666666666666},
 {'Light cocktail martini': {('alco', 'soft', 'bw'): [],
   ('alco', 'soft'): [],
   ('alco',): ['red martini'],
   ('noalco', 'fruits', 'juices', 'j_sweet'): [],
   ('noalco', 'fruits', 'juices', 'j_citric'): [],
   ('noalco', 'fruits', 'solid', 's_sweet'): [],
   ('noalco', 'fruits', 'solid', 's_citric'): ['lime'],
   ('noalco', 'veggies'): ['mint'],
   ('noalco', 'fruits', 'syrups', 'j_sweet'): [],
   ('te', 'sweet'): ['sugar'],
   ('te',): [],
   ('others',): ['sparkling mineral water']},
  'Cocktail appetizer': {('alco', 'soft', 'bw'): ['dry white wine'],
   ('alco', 'soft'): [],
   ('alco',): ['rum'],
   ('noalco', 'fruits', 'juices', 'j_sweet'): [],
   ('noalco', 'fruits', 'juices', 'j_citric'): [],
   ('noalco', 'fruits', 'solid', 's_sweet'): [],
   ('noalco', 'fruits', 'solid', 's_citric'): ['lemon', 'orange'],
   ('noalco', 'veggies'): [],


## Adaptation

{'liquid': [('alcohol', 'red martini')],
 'noalco': [('fruits', 'citric', 'solid', 'lime'), ('veggies', 'mint')],
 'te': [('sweet', 'sugar')],
 'others': [('sparkling mineral water',)]}

In [231]:
from pprint import pprint
q_ingredients_list = q
preference = 'alco'

res = {}
categories = list(taxonomy_scheme.keys())
for title in cocktail_matches.keys():
    cocktail = deepcopy(cocktail_matches[title])
    print('q')
    pprint(q)
    print('cocktail')
    pprint(cocktail)
    
    print()
    matches = ingredients_list.copy()
    sim_matches = 0
    for q_cat in categories:
        q_general_cat = q_cat[0]
        
        w = df_preferences[q_cat][preference]
        for q_ingredient in q_ingredients_list[q_cat]:
            print(q_ingredient)
            q_ingredient_cat = tuple(list(q_cat) + [q_ingredient])
            c_matches = 0

            c_same_general_cat = 0
            sim_general_matches = np.array([])
            d_ingredient_cat_lst = []
            q_ingredient_cat_lst = []
            # if the same ingredient don't bother
            if q_ingredient in cocktail[q_cat]:
                cocktail[q_cat].remove(q_ingredient)
                pprint(cocktail)
                print()
                continue
            for d_cat in categories:
                d_general_cat = d_cat[0]
                for d_ingredient in cocktail[d_cat]:
                    if d_general_cat == q_general_cat:
                        q_ingredient_cat_lst.append(q_ingredient_cat)
                        d_ingredient_cat = tuple(list(d_cat) + [d_ingredient])
                        d_ingredient_cat_lst.append(d_ingredient_lst)
                        sim_score = taxonomy_score(q_ingredient_cat, d_ingredient_cat) * w
                        sim_general_matches = np.append(sim_general_matches, sim_score)
            if d_ingredient_lst:
                argmax_sim = sim_general_matches.argmax()
                max_sim = sim_general_matches[argmax_sim]
                print()
            else:
                print('empty', q_ingredient)
''' for each q_cat =>  
    q_cat:[((tobechanged_list_ordered, sim), from_which)] for the same tobechanged maxsim
'''

q
{('liquid', 'alcohol'): ['red martini'],
 ('liquid', 'alcohol', 'soft'): [],
 ('liquid', 'alcohol', 'soft', 'bw'): [],
 ('liquid', 'juices'): ['lemon juice'],
 ('liquid', 'juices', 'citric'): [],
 ('others',): [],
 ('solid', 'fruits'): [],
 ('solid', 'fruits', 'citric'): ['lime', 'orange'],
 ('solid', 'veggies'): [],
 ('te',): [],
 ('te', 'sweet'): ['sugar'],
 ('te', 'syrups'): []}
cocktail
{('liquid', 'alcohol'): ['red martini'],
 ('liquid', 'alcohol', 'soft'): [],
 ('liquid', 'alcohol', 'soft', 'bw'): [],
 ('liquid', 'juices'): [],
 ('liquid', 'juices', 'citric'): [],
 ('others',): ['sparkling mineral water'],
 ('solid', 'fruits'): [],
 ('solid', 'fruits', 'citric'): ['lime'],
 ('solid', 'veggies'): ['mint'],
 ('te',): [],
 ('te', 'sweet'): ['sugar'],
 ('te', 'syrups'): []}

red martini
{('liquid', 'alcohol'): [],
 ('liquid', 'alcohol', 'soft'): [],
 ('liquid', 'alcohol', 'soft', 'bw'): [],
 ('liquid', 'juices'): [],
 ('liquid', 'juices', 'citric'): [],
 ('others',): ['sparkling mi

In [79]:
len(unique_ingredients), unique_ingredients

(156,
 {'amber rum',
  'angostura bitter',
  'anise basil',
  'apple',
  'apple cider',
  'apple juice',
  'apple liqueur',
  'apricot',
  'apricot juice',
  'apricot liqueur',
  'banana',
  'banana juice',
  'beer',
  'benedictine',
  'berry',
  'berry juice',
  'blackcurrant',
  'blood orange',
  'blue curacao',
  'blush wine',
  'brown sugar',
  'brown_sugar',
  'cachaca',
  'calvados',
  'campari',
  'cane sugar',
  'cane sugar syrup',
  'cava',
  'celery salt',
  'champagne',
  'cherry',
  'cherry juice',
  'cinnamon',
  'citrus fruit',
  'clementine orange',
  'coca-cola',
  'coconut',
  'coconut milk',
  'coffee',
  'coffee liqueur',
  'cognac',
  'cointreau',
  'coriander',
  'cranberry juice',
  'cremant',
  'creme de cafe',
  'creme de cassis',
  'crushed ice',
  'cucumbers',
  'currant',
  'currant syrup',
  'dark rum',
  'dry white wine',
  'egg',
  'fruit',
  'fruit syrup',
  'get_27',
  'gin',
  'ginger',
  'grand marnier',
  'granulated sugar',
  'grape juice',
  'grapef

## Check similarity with wordnet

In [167]:
words_found = []
words_notfound = []
for w in unique_ingredients:
    syn = wn.synsets(w)
    if not syn:
        words_notfound.append(w)
    else:
        words_found.append(w)


In [168]:
len(words_notfound), len(words_found)

(102, 55)

In [169]:
# split the multi-word ingredients
ingredient_words = []
for ingr in all_ingredients:
    if '_' in ingr:
        print(ingr)
        ingr = ingr.replace('_',' ')
    if ingr == 'get_27':
        ingr = 'get27'
    elif ingr == 'manguo juice':
        ingr = 'mango juice'
    #elif ingr == 'cachaça':
    #    ingr = 'cachaca'
    ingredient_words.extend(ingr.split())

noilly_prat
brown_sugar
get_27


In [170]:
def w2v_get_weights(words, n_words, emb_dim, w2v_model):
    c = 0
    not_found = []
    embedding_matrix = np.zeros((n_words, emb_dim))
    for i, word in enumerate(words):
        try:
            embedding_vector = w2v_model.wv[word]
        except KeyError:
            embedding_vector = None
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
        else:
            c += 1
            not_found.append(word)
    print(c, 'were not found')
    return embedding_matrix, not_found

In [171]:
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

dir_folder = 'glove'
glove_name = 'glove_42B_300d.txt'
weights_file = path.join(dir_folder, glove_name.split('.')[0] + '.npy')
glove_file = path.join(dir_folder, glove_name)
emb_file = '2' + glove_name
w2v_glove_file = path.join(dir_folder, emb_file)


if not path.exists(w2v_glove_file):
    glove2word2vec(glove_file, w2v_glove_file)

if not path.exists(weights_file):
    w2v_model = KeyedVectors.load_word2vec_format(w2v_glove_file, binary=False)
    emb_weights, not_found = w2v_get_weights(ingredient_words,
                                len(ingredient_words),
                                300,
                                w2v_model)
    del w2v_model
    np.save(path.join(dir_folder,weights_file), emb_weights)
else:
    emb_weights = np.load(weights_file)

In [ ]:
# print embedding file
embeddings_index = {}
with open(path.join(dir_folder, glove_name)) as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

In [173]:
emb_weights.shape

(781, 300)